#### Тестовое задание 2 (часть 1)
Соколова Ирина
09/10/2020


### Найти паттерны товаров

В файле содержится информация о покупках людей

id	Товар	Количество
1	Арбуз	1.0
1	Чай	1.0
1	Сгущёнка	0.5
2	Арбуз	3.0
2	Чай	1.0

id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
Товар – наименование товара
Количество – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

1_Товар	2_Товар	Встречаемость
Чай	Арбуз	2
Арбуз	Сгущёнка	1
Чай	Сгущёнка	1
1_Товар – наименование первого товара
2_Товар – наименование второго товара
Встречаемость – число раз, когда такая пара была встречена

Другими словами: 2 раза люди покупали одновременно чай и арбуз, 1 раз одновременно покупали арбуз и сгущёнку и 1 раз одновременно были куплены чай со сгущёнкой.

Напишите код на python для получения нужной таблицы и укажите 5 наиболее распространённых паттернов.

In [1]:
#загрузка данных
file = 'https://stepik.org/media/attachments/lesson/409319/test1_completed.csv'

In [2]:
#библиотеки
import pandas as pd
import numpy as np
import itertools

In [3]:
df = pd.read_csv(file)

In [4]:
df.head()

,id,Товар,Количество
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


In [5]:
#посмотрим на продуктовые наборы в каждой покупке
df_ = df[['id', 'Товар']].groupby('id')['Товар'].value_counts().to_frame()
df_.head()

Товар
id    Товар                 
17119 Лимон                1
      Лимон оранжевый      1
      Лук репчатый         1
      Лук-порей            1
      Малина свежая        1

In [6]:
df_['Товар'].unique()

array([1])

In [7]:
#объединим каждый набор в список относящийся к каждой покупке
df__ = df_.unstack(level=1).reset_index().melt(id_vars=['id']) \
    .query("value == 1").groupby('id')['Товар'].apply(list).reset_index()

In [8]:
df__.head()

,id,Товар
0,17119,"[Лимон, Лимон оранжевый, Лук репчатый, Лук-пор..."
1,17530,"[Бразильский орех, Дыня Гуляби, Изюм Голд, Изю..."
2,17618,"[Абрикосы молдавские, Апельсины столовые, Гран..."
3,17724,"[Абрикосы молдавские, Арбуз, Галангал корень, ..."
4,17814,"[Арбуз, Вишня садовая, Груша Аббат, Груша Пакх..."


### Функция для поиска паттернов
Имея таким образом подготовленный DataFrame можно написать функцию для опреденения паттернов в покупках

1) Используем itertools.combinations для получения всех возможных комбинаций пар и преобразования результирующего ряда в представление списка, которое должно быть передано конструктору dataframe позже.

2) Сложим DF и возьмите соответствующий Series.value_counts() . Возьмем индекс с наибольшим количеством, используя Series.idxmax() .

In [9]:
def basket_pattern(column: pd.core.series.Series,  num_of_common: int, num_of_comb: int):
    comb = lambda x: list(itertools.combinations(x, num_of_common))
    L = column.map(comb).tolist()
    Pair_basket =  pd.DataFrame(L).stack().value_counts(sort=True).to_frame().reset_index().rename(columns = {'index':'pattern', 0 : 'numb'})
    return Pair_basket.head(num_of_comb)    

In [10]:
basket_pattern_2 = basket_pattern(df__['Товар'], 2, 10)
basket_pattern_2.head()

,pattern,numb
0,"(Огурцы Луховицкие, Укроп)",431
1,"(Петрушка, Укроп)",408
2,"(Арбуз, Огурцы Луховицкие)",345
3,"(Кабачки, Огурцы Луховицкие)",326
4,"(Кинза, Укроп)",303


In [11]:
basket_pattern_5 = basket_pattern(df__['Товар'], 3, 5)
print (basket_pattern_5)

                                    pattern  numb
0      (Огурцы Луховицкие, Петрушка, Укроп)   232
1         (Кинза, Огурцы Луховицкие, Укроп)   181
2            (Лук зеленый, Петрушка, Укроп)   175
3                  (Кинза, Петрушка, Укроп)   164
4  (Лук репчатый, Огурцы Луховицкие, Укроп)   164


### Вывод

В задании требовалось найти пять паттернов наиболее покупаемых пар товаров. И написать код, который будет возвращать таблицу.

Функция basket_pattern(column: pd.core.series.Series,  num_of_common: int, num_of_comb: int) требует на вход pd.Series каждая строчка которой представляет собой лист состоящий из покупки одного клиента с перечнем товаров, 
затем количество товаровб которые покупаю чаще всего вместе (в задании 2), сколько паттернов нужно отобразить в таблице.

Наши победители:

In [12]:
basket_pattern_2.head()

,pattern,numb
0,"(Огурцы Луховицкие, Укроп)",431
1,"(Петрушка, Укроп)",408
2,"(Арбуз, Огурцы Луховицкие)",345
3,"(Кабачки, Огурцы Луховицкие)",326
4,"(Кинза, Укроп)",303
